## Tutorial 2 smolagent
https://huggingface.co/docs/smolagents/guided_tour?Pick+a+LLM=HF+Inference+API

NOTA: Recomendable ejecutarlo en un entorno virtual de python

In [ ]:
!pip install smolagents python-dotenv 
#transformers accelerate

In [ ]:
!python.exe -m pip install --upgrade pip

In [1]:
from dotenv import load_dotenv
import os

# Cargar las variables de entorno del archivo .env
load_dotenv()

# Acceder a la clave privada
hf_key = os.getenv("HF_TOKEN")
#!huggingface-cli login --token $hf_key

In [2]:
model_id = "meta-llama/Llama-3.3-70B-Instruct" 
# model_id = "Qwen/Qwen2.5-Coder-32B-Instruct"
# model_id = "meta-llama/Llama-3.2-3B-Instruct"
# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

In [6]:
from smolagents import CodeAgent, HfApiModel

model = HfApiModel(model_id=model_id)

### Utilidades

1. Buscar en internet conDuckDuckGo

In [7]:
from smolagents import DuckDuckGoSearchTool

search_tool = DuckDuckGoSearchTool()
print(search_tool("Who's the current president of Russia?"))

## Search Results

[List of presidents of Russia - Wikipedia](https://en.wikipedia.org/wiki/List_of_presidents_of_Russia)
The office of the president of Russia is the highest authority in the Russian Federation.The holder is the federation's head of state and has formal presidency over the State Council as well as being the commander in chief of the Russian Armed Forces.The office was introduced in 1918 after the February Revolution with the current office emerging after a referendum of 1991. [1]

[President of Russia - Wikipedia](https://en.wikipedia.org/wiki/President_of_Russia)
The president of Russia, officially the president of the Russian Federation (Russian: Президент Российской Федерации, romanized: Prezident Rossiyskoy Federatsii), is the executive head of state of Russia.The president is the chair of the Federal State Council and the supreme commander-in-chief of the Russian Armed Forces.It is the highest office in Russia.

[Biographies of presidents of Russia - President of 

2. Crear una tool para obtener el título de una página web

La siguiente versión a mi no me funciona:

In [8]:
from smolagents import ToolCallingAgent

agent = ToolCallingAgent(tools=[], model=model)
agent.run("Could you get me the title of the page at url 'https://huggingface.co/blog'?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could you get me the title of the page at url 'https://huggingface.co/blog'?                                    │
│                                                                                                                 │
╰─ HfApiModel - meta-llama/Llama-3.3-70B-Instruct ────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'No function available to get the title of the          │
│ webpage'}                                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: No function available to get the title of the webpage

[Step 0: Duration 0.44 seconds| Input tokens: 1,071 | Output tokens: 26]

'No function available to get the title of the webpage'

Pero si se define como una tool, entonces si funciona:

In [10]:
!pip install bs4 BeautifulSoup4

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [10]:
import requests
from bs4 import BeautifulSoup
from smolagents import tool, CodeAgent, HfApiModel

# Definir la función para obtener el título de la página web
@tool
def get_title(url: str) -> str:
    '''
    Obtener el título de una página web

    Args:
        url: Url de la página web
    '''
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title_tag = soup.find('title')
    return title_tag.text if title_tag else "No title found"

# Crear la lista de herramientas con la función de obtener título
tools = [
    {
        "name": "get_title",
        "function": get_title,
        "description": "Fetches the title of a webpage given a URL."
    }
]

# Inicializar el agente en modo HFApiMode
model = HfApiModel(model_id=model_id, token=hf_key) # You can choose to not pass any model_id to HfApiModel to use a default free model
# you can also specify a particular provider e.g. provider="together" or provider="sambanova"
agent = CodeAgent(tools=[get_title], model=model, add_base_tools=True)

# Ejecutar el agente con la solicitud
result = agent.run("Could you get me the title of the page at url 'https://huggingface.co/blog'?")
print(result)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could you get me the title of the page at url 'https://huggingface.co/blog'?                                    │
│                                                                                                                 │
╰─ HfApiModel - meta-llama/Llama-3.3-70B-Instruct ────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  title = get_title(url='https://huggingface.co/blog')                                                             
  print(title)                                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Hugging Face – Blog

Out: None

[Step 0: Duration 5.24 seconds| Input tokens: 2,178 | Output tokens: 66]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Hugging Face – Blog")                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Hugging Face – Blog

[Step 1: Duration 3.54 seconds| Input tokens: 4,512 | Output tokens: 135]

Hugging Face – Blog
